# Projeto End-to-End: Fine-tuning de YOLO (Ultralytics) para Detecção de Capacete (PPE)


## 1. Objetivos de Aprendizagem

Ao final deste projeto, você será capaz de:
*   **Versionamento**: Gerenciar versões de datasets de forma profissional.
*   **Auditoria e QA**: Identificar e corrigir labels ruidosas antes de treinar.
*   **Augmentations**: Criar pipelines de aumento de dados que respeitam a distribuição física do problema.
*   **Transfer Learning**: Adaptar pesos pré-treinados no COCO para um domínio específico (EPIs).
*   **Avaliação**: Ir além do mAP e fazer análise de erros com matriz de confusão e visualização de *false positives*.
*   **HPO**: Otimizar hiperparâmetros (LR, Momentum, Decay) para extrair a última gota de performance.

## 2. Visão Geral do Pipeline

```ascii
[Internet/Source] 
      ⬇
[Dataset Raw] ➡ [Auditoria/Cleaning] ➡ [Dataset Gold v1.0]
                                             ⬇
                                    [Data Augmentation (Albumentations)]
                                             ⬇
                                    [Dataset Ready v2.0]
                                             ⬇
[YOLO11 Pre-trained] ➡ [Baseline Train] ➡ [Eval Baseline] ➡ [Error Analysis]
                                                                     ⬇
                                                         [HPO (Optuna) - Tuning]
                                                                     ⬇
                                                         [Training Final (Best Params)]
                                                                     ⬇
                                                         [Export ONNX/TensorRT] ➡ [Inference demo]
```

## 3. Conceitos Chave

### O que é Transfer Learning no contexto YOLO?
Em vez de iniciar o treinamento com pesos aleatórios (o que exigiria milhões de imagens e dias de treino), usamos **pesos pré-treinados** em datasets massivos (como o COCO, que tem 80 classes gerais).
No **Fine-tuning**, "congelamos" (ou treinamos com LR baixo) as primeiras camadas que detectam características básicas (bordas, texturas) e treinamos agressivamente apenas as últimas camadas (Head) para reconhecer nossas novas classes (Capacete, Colete, Sem Capacete).

> [!IMPORTANT]
> **Regras de Ouro**
> 1.  **Nunca vaze dados**: O conjunto de Teste é sagrado. Nunca olhe para ele durante o tuning de hiperparâmetros. Use Validação para isso.
> 2.  **Meça antes de otimizar**: Não mude augmentation ou modelo sem ter um baseline sólido.
> 3.  **Garbage In, Garbage Out**: 1 hora limpando o dataset vale mais que 10 horas tunando learning rate. A qualidade do label é o teto da sua performance.
> 4.  **Distribuição importa**: Augmentations devem simular variações reais. Se sua câmera é fixa, não faz sentido fazer rotações de 180 graus.




### 2.1. Contextualização: Visão Clássica vs Deep Learning

Antes de mergulhar no treinamento, é crucial entender *por que* estamos usando Deep Learning (YOLO) e não técnicas clássicas.

| Aspecto | Visão Clássica (OpenCV/Geometria) | Deep Learning (YOLO/CNNs) |
| :--- | :--- | :--- |
| **Foco** | Geometria, Medição Exata, Bordas | Semântica, Classificação, Robustez |
| **Exemplo** | Medir o diâmetro de um parafuso em micrômetros. | Detectar se um operário está usando capacete. |
| **Vantagem** | Determinístico, Baixa Latência, Explica o "Como". | Generaliza bem, ignora ruído, ignora iluminação variável. |
| **Case Real** | **Metrologia Industrial**: Garantir tolerância de peças. <br> **Visual SLAM**: Robôs que mapeiam túneis sem GPS. <br> **Leitura de Código de Barras**: Decodificação de bits 0/1. | **Segurança**: PPE, Detecção de Armas. <br> **Autônomos**: Detectar pedestres/carros. <br> **Médico**: Detectar tumores em Raio-X. |

> **Resumo**: Use **Deep Learning** para responder "O que é isso?". Use **Visão Clássica** para responder "Qual o tamanho exato disso?" ou "Onde isso está exatamente?".


## 4. Setup do Ambiente

Antes de começar, precisamos garantir que o ambiente está pronto. PyTorch e YOLO dependem fortemente da GPU para serem viáveis.
Checar a VRAM é vital para decidir o `batch_size`. Na RTX 4060 (8GB), modelos `Nano` e `Small` rodam folgados. `Medium` exige cuidado.



In [36]:
# Instalação de dependências
# Usamos 'uv pip' se disponível para velocidade, ou pip padrão.
# A flag -q (quiet) reduz o output.
import sys
import subprocess

def install_packages(packages):
    print(f"Instalando: {', '.join(packages)}...")
    # Tenta usar uv se estiver no path, senao usa pip module
    try:
        subprocess.check_call(["uv", "pip", "install"] + packages)
    except FileNotFoundError:
        subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)

# Lista de libs essenciais para o projeto
libs = [
    "ultralytics",       # YOLO11 framework
    "opencv-python",     # Visão computacional básica
    "matplotlib",        # Gráficos
    "numpy", 
    "pandas",            # Análise de logs
    "pyyaml",            # Configs
    "tqdm",              # Barras de progresso
    "albumentations",    # Augmentations avançados
    "optuna",            # HPO
    "supervision"        # Visualização e Utilitários extras
    "mlflow",            # Experiment Tracking
]

# Descomente a linha abaixo para instalar (pode demorar um pouco na primeira vez)
# install_packages(libs)
print("Dependências verificadas.")


Dependências verificadas.


In [37]:
import json
import torch
import ultralytics
import cv2
import numpy as np
import pandas as pd
import matplotlib
import os
import random
import shutil

# Configurações de exibição
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Verificações de Versão e Hardware
print(f"PyTorch Version: {torch.__version__}")
print(f"Ultralytics Version: {ultralytics.__version__}")
print(f"OpenCV Version: {cv2.__version__}")

# Check CUDA
if torch.cuda.is_available():
    print(f"CUDA Disponível: Sim")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = 'cuda'
else:
    print("CUDA NÃO DETECTADO. O treinamento será extremamente lento.")
    device = 'cpu'


PyTorch Version: 2.9.0+cu130
Ultralytics Version: 8.3.253
OpenCV Version: 4.12.0
CUDA Disponível: Sim
GPU: NVIDIA GeForce RTX 3050 6GB Laptop GPU
VRAM Total: 6.09 GB


In [38]:
# Reprodutibilidade (Seeds)
# Embora operações em GPU tenham algum não-determinismo inerente, fixar seeds ajuda na consistência.
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # torch.backends.cudnn.deterministic = True # Pode deixar lento
        # torch.backends.cudnn.benchmark = False

SEED = 42
set_seed(SEED)
print(f"Seed fixada em: {SEED}")

Seed fixada em: 42


In [39]:
import pathlib
# Configuração de Diretórios do Projeto
# É fundamental manter o workspace organizado para não misturar experimentos.

BASE_DIR = os.path.abspath("./workspace")
DATA_ROOT = os.path.join(BASE_DIR, "datasets")
RUNS_DIR = os.path.join(BASE_DIR, "runs")
ARTIFACTS_DIR = os.path.join(BASE_DIR, "artifacts") # Pesos finais, onnx, etc
REPORTS_DIR = os.path.join(BASE_DIR, "reports")     # Gráficos e análises de erro

dirs = [DATA_ROOT, RUNS_DIR, ARTIFACTS_DIR, REPORTS_DIR]

for d in dirs:
    os.makedirs(d, exist_ok=True)
    print(f"Diretório garantido: {d}")

# Variáveis Globais de Configuração
PROJECT_NAME = "yolo_ppe_finetune"

# Configuração do MLflow
import mlflow

# Define onde os logs 'oficiais' do MLflow ficam.
# Pode ser local (./mlruns) ou um servidor remoto (http://...)
mlflow.set_tracking_uri(pathlib.Path(os.path.join(BASE_DIR, "mlruns")).as_uri())
mlflow.set_experiment(PROJECT_NAME)
print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"MLflow Experiment: {PROJECT_NAME}")


Diretório garantido: /home/estevaosilva/PycharmProjects/NCIA/AULAS/Aula_58/Projeto_visao/workspace/datasets
Diretório garantido: /home/estevaosilva/PycharmProjects/NCIA/AULAS/Aula_58/Projeto_visao/workspace/runs
Diretório garantido: /home/estevaosilva/PycharmProjects/NCIA/AULAS/Aula_58/Projeto_visao/workspace/artifacts
Diretório garantido: /home/estevaosilva/PycharmProjects/NCIA/AULAS/Aula_58/Projeto_visao/workspace/reports
MLflow Tracking URI: file:///home/estevaosilva/PycharmProjects/NCIA/AULAS/Aula_58/Projeto_visao/workspace/mlruns
MLflow Experiment: yolo_ppe_finetune


## 5. Preparação do Dataset (PPE - Personal Protective Equipment)

Para este projeto, usaremos um dataset de trabalhadores ('Hard Hat Workers') contendo 3 classes principais:
1.  `helmet` (Capacete)
2.  `vest` (Colete reflexivo)
3.  `person` (Pessoa)

> **Nota sobre Estrutura YOLO**:
> O framework Ultralytics espera uma estrutura rígida para evitar configurações manuais complexas.
> - `dataset/images/train`, `dataset/images/val`
> - `dataset/labels/train`, `dataset/labels/val`
>
> Se seus labels não estiverem na pasta paralela exata, o treino falhará silenciosamente ou dará erro de "no labels found".



In [40]:
# Opção A: Download Automático (Exemplo com dataset público no formato YOLO)
dataset_url = "https://github.com/gradio-app/gradio/raw/main/demo/yolo_detection/files/ppe_data.zip" 

def download_dataset(url, dest_dir):
    import urllib.request
    import zipfile
    
    zip_path = os.path.join(dest_dir, "dataset.zip")
    print(f"Baixando dataset para {zip_path}...")
    try:
        os.makedirs(dest_dir, exist_ok=True)
        urllib.request.urlretrieve(url, zip_path)
        print("Download completo. Extraindo...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(dest_dir)
        print("Extração concluída.")
    except Exception as e:
        print(f"Erro no download automátio: {e}")
        print("--> USE A OPÇÃO B (Manual) <--")

In [41]:
# Opção B: Upload Manual (Fallback Inteligente)
# Procura por zips comuns na raiz ou na pasta de datasets

possible_zips = [
    os.path.join(DATA_ROOT, "hardhat_raw.zip"),
    os.path.join(BASE_DIR, "..", "ppe_v1.zip"),         # Na raiz do projeto
    os.path.join(BASE_DIR, "..", "hardhat_raw.zip"),    # Na raiz do projeto
]

zip_path = None
for p in possible_zips:
    if os.path.exists(p):
        zip_path = p
        break

if zip_path:
    # Se achou na raiz, copia para o lugar certo
    target_zip = os.path.join(DATA_ROOT, "hardhat_raw.zip")
    if zip_path != target_zip:
        print(f"Encontrado zip em {zip_path}. Copiando para {target_zip}...")
        shutil.copy2(zip_path, target_zip)
    else:
        print(f"Zip encontrado em {zip_path}.")
        
    # Atualiza a variável para a célula de extração usar (se necessário)
    zip_manual_path = target_zip
else:
    print("Nenhum zip encontrado automaticamente.")
    print(f"Por favor, coloque 'hardhat_raw.zip' ou 'ppe_v1.zip' em: {DATA_ROOT}")


Nenhum zip encontrado automaticamente.
Por favor, coloque 'hardhat_raw.zip' ou 'ppe_v1.zip' em: /home/estevaosilva/PycharmProjects/NCIA/AULAS/Aula_58/Projeto_visao/workspace/datasets


In [42]:
# Estruturação e Limpeza (Standard YOLO)
# Objetivo: mover tudo para ./workspace/datasets/ppe_v1/{images,labels}/{train,val,test}

RAW_DIR = os.path.join(DATA_ROOT, "raw_extract") # Pasta temporária
FINAL_DATASET_DIR = os.path.join(DATA_ROOT, "ppe_v1")

def organize_yolo_structure(source, dest):
    train_dir_check = os.path.join(dest, "images", "train")
    if os.path.exists(train_dir_check) and len(os.listdir(train_dir_check)) > 50:
        print(f"Dataset destino já existe e parece válido ({len(os.listdir(train_dir_check))} imagens no treino). Pulando organização.")
        return
    elif os.path.exists(dest):
        print(f"Dataset existe mas parece vazio/incompleto. Re-organizando...")
        # shutil.rmtree(dest) # Opcional: limpar antes


    print(f"Organizando {source} -> {dest} ...")
    os.makedirs(dest, exist_ok=True)
    
    # Criar subpastas padrão YOLO
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(dest, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(dest, 'labels', split), exist_ok=True)
    
    images = []
    for root, _, files in os.walk(source):
        for f in files:
            if f.lower().endswith(('.jpg', '.jpeg', '.png')):
                images.append(os.path.join(root, f))
                
    random.shuffle(images)
    split_idx = int(len(images) * 0.8)
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]
    
    def move_files(file_list, split_name):
        for img_path in file_list:
            shutil.copy(img_path, os.path.join(dest, 'images', split_name, os.path.basename(img_path)))
            
            label_path = os.path.splitext(img_path)[0] + ".txt"
            if os.path.exists(label_path):
                shutil.copy(label_path, os.path.join(dest, 'labels', split_name, os.path.basename(label_path)))
            else:
                with open(os.path.join(dest, 'labels', split_name, os.path.splitext(os.path.basename(img_path))[0] + ".txt"), 'w') as f:
                    pass

    move_files(train_imgs, 'train')
    move_files(val_imgs, 'val')
    print("Organização completa.")

In [43]:
# Gerar data.yaml (A Identidade do Dataset)
yaml_content = f'''
path: {FINAL_DATASET_DIR} # dataset root dir
train: images/train  # train images (relative to 'path') 
val: images/val      # val images (relative to 'path')
test:  # test images (optional)

names:
  0: helmet
  1: vest
  2: person
'''

yaml_path = os.path.join(FINAL_DATASET_DIR, "data.yaml")

# Check diretório antes de escrever (Bug fix)
os.makedirs(FINAL_DATASET_DIR, exist_ok=True)

if not os.path.exists(yaml_path):
    with open(yaml_path, 'w', encoding='utf-8') as f:
        f.write(yaml_content)
    print(f"data.yaml criado em: {yaml_path}")
else:
    print(f"data.yaml já existe em: {yaml_path}")
    
# Mostrar conteúdo
print("--- CONTEÚDO DO DATA.YAML ---")
with open(yaml_path, 'r', encoding='utf-8') as f:
    print(f.read())
print("-----------------------------")

data.yaml já existe em: /home/estevaosilva/PycharmProjects/NCIA/AULAS/Aula_58/Projeto_visao/workspace/datasets/ppe_v1/data.yaml
--- CONTEÚDO DO DATA.YAML ---

path: /home/estevaosilva/PycharmProjects/NCIA/AULAS/Aula_58/Projeto_visao/workspace/datasets/ppe_v1 # dataset root dir
train: images/train  # train images (relative to 'path') 
val: images/val      # val images (relative to 'path')
test:  # test images (optional)

names:
  0: helmet
  1: vest
  2: person

-----------------------------


In [44]:
# Validação Básica
def validate_dataset(path):
    print(f"Validando dataset em: {path}")
    for split in ['train', 'val']:
        img_dir = os.path.join(path, 'images', split)
        lbl_dir = os.path.join(path, 'labels', split)
        
        if not os.path.exists(img_dir):
            print(f"⚠️ Split {split} não encontrado!")
            continue
            
        n_imgs = len(os.listdir(img_dir))
        n_lbls = len(os.listdir(lbl_dir))
        
        print(f"[{split.upper()}] Imagens: {n_imgs} | Labels: {n_lbls}")
        
        if n_imgs != n_lbls:
            print(f"   🔴 ALERTA: Número de imagens e labels difere!")
        elif n_imgs == 0:
            print(f"   🔴 ALERTA: Split vazio!")
        else:
            print(f"   🟢 OK.")

# validate_dataset(FINAL_DATASET_DIR)



## 6. Auditoria e Limpeza do Dataset

"Dados Limpos > Modelos Complexos".
Antes de treinar, precisamos garantir que o dataset não tem lixo.
Erros comuns em datasets de detecção:
1.  **Orphans**: Imagem sem txt ou txt sem imagem.
2.  **Degenerate BBoxes**: Boxes com largura/altura = 0 ou muito pequenas.
3.  **Out of Bounds**: Coordenadas > 1.0 ou < 0.0 (normalização errada).
4.  **Class IDs**: Classes que não existem no `names` (ex: id 5 num dataset de 3 classes).



In [45]:
# Funções de Validação e Métricas
import glob

def verify_yolo_label(lbl_path, num_classes=3):
    issues = []
    try:
        if os.path.getsize(lbl_path) == 0:
            return ["empty_file"] # Label vazio é valido (sem objetos), mas bom saber.
            
        with open(lbl_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            
        for i, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) != 5:
                issues.append(f"Line {i}: FormatErr (not 5 cols)")
                continue
                
            cls, x, y, w, h = map(float, parts)
            
            if not (0 <= int(cls) < num_classes):
                issues.append(f"Line {i}: BadClassId ({int(cls)})")
            
            if not (0 <= x <= 1 and 0 <= y <= 1 and 0 <= w <= 1 and 0 <= h <= 1):
                issues.append(f"Line {i}: OutOfBounds ({x},{y},{w},{h})")
                
            if w <= 0 or h <= 0:
                issues.append(f"Line {i}: Degenerate ({w},{h})")
                
    except Exception as e:
        issues.append(f"ReadError: {str(e)}")
        
    return issues

def find_orphans(img_dir, lbl_dir):
    # Procura descasamentos entre imagens e labels
    imgs = {os.path.splitext(f)[0] for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))}
    lbls = {os.path.splitext(f)[0] for f in os.listdir(lbl_dir) if f.endswith('.txt')}
    
    img_orphans = imgs - lbls # Imagens sem labels (pode ser intencional bg-only, mas vale avisar)
    lbl_orphans = lbls - imgs # Labels sem imagens (erro grave, deletar)
    
    return img_orphans, lbl_orphans



In [46]:
# Rotina Clean Dataset (Move para Quarentena)
QUARANTINE_DIR = os.path.join(DATA_ROOT, "quarantine")
os.makedirs(QUARANTINE_DIR, exist_ok=True)

def clean_dataset(dataset_path):
    report = []
    print(f"Iniciando limpeza silenciosa em: {dataset_path}")
    print("Verificando integridade de imagens e labels... (isso pode levar alguns segundos)")
    
    for split in ['train', 'val']:
        img_dir = os.path.join(dataset_path, 'images', split)
        lbl_dir = os.path.join(dataset_path, 'labels', split)
        
        if not os.path.exists(img_dir): continue

        # 1. Checar Orfãos
        img_orphans, lbl_orphans = find_orphans(img_dir, lbl_dir)
        
        # Labels orfãs -> Quarentena
        for o in lbl_orphans:
            src = os.path.join(lbl_dir, o + ".txt")
            dst = os.path.join(QUARANTINE_DIR, "orphans", split, o + ".txt")
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.move(src, dst)
            report.append({'file': o, 'split': split, 'issue': 'Orphan Label (Moved)'})
            
        # 2. Validar Conteúdo
        lbl_files = glob.glob(os.path.join(lbl_dir, "*.txt"))
        for lf in lbl_files:
            issues = verify_yolo_label(lf)
            if issues:
                base = os.path.basename(lf)
                serious = [i for i in issues if i != "empty_file"]
                if serious:
                    dst_lbl = os.path.join(QUARANTINE_DIR, "corrupt", split, "labels", base)
                    os.makedirs(os.path.dirname(dst_lbl), exist_ok=True)
                    shutil.move(lf, dst_lbl)
                    
                    base_img = os.path.splitext(base)[0] + ".jpg" 
                    src_img = os.path.join(img_dir, base_img)
                    if os.path.exists(src_img):
                        dst_img = os.path.join(QUARANTINE_DIR, "corrupt", split, "images", base_img)
                        os.makedirs(os.path.dirname(dst_img), exist_ok=True)
                        shutil.move(src_img, dst_img)
                    
                    report.append({'file': base, 'split': split, 'issue': "; ".join(serious)})

    print("-" * 30)
    print(f"Limpeza concluída.")
    print(f"Total de arquivos problemáticos movidos: {len(report)}")
    if len(report) > 0:
        rep_path = os.path.join(REPORTS_DIR, 'audit_report.csv')
        pd.DataFrame(report).to_csv(rep_path)
        print(f"Relatório detalhado salvo em: {rep_path}")
        print("Arquivos corrompidos foram movidos para a pasta 'quarantine'.")
    else:
        print("Nenhum problema crítico encontrado. Dataset limpo!")
    print("-" * 30)

# Executar Limpeza
# clean_dataset(FINAL_DATASET_DIR)


In [47]:
# Visualização de Amostras
# Vamos desenhar as caixas para ver se faz sentido visualmente.

def plot_samples(dataset_path, split='train', n=9):
    img_dir = os.path.join(dataset_path, 'images', split)
    lbl_dir = os.path.join(dataset_path, 'labels', split)
    
    all_imgs = os.listdir(img_dir)
    if len(all_imgs) == 0: return
    
    samples = random.sample(all_imgs, min(n, len(all_imgs)))
    
    plt.figure(figsize=(15, 10))
    for i, img_file in enumerate(samples):
        img_path = os.path.join(img_dir, img_file)
        lbl_path = os.path.join(lbl_dir, os.path.splitext(img_file)[0] + ".txt")
        
        img = cv2.imread(img_path)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h_img, w_img, _ = img.shape
        
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r', encoding='utf-8') as f:
                for line in f:
                    c, x, y, w, h = map(float, line.split())
                    x1 = int((x - w/2) * w_img)
                    y1 = int((y - h/2) * h_img)
                    x2 = int((x + w/2) * w_img)
                    y2 = int((y + h/2) * h_img)
                    
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img, str(int(c)), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
        
        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(img_file)
    plt.tight_layout()
    plt.show()

# plot_samples(FINAL_DATASET_DIR)



## 7. Como Rotular Corretamente (Guidelines)

Se a auditoria mostrar problemas, ou se você for rotular novos dados, siga este padrão:

### Definições Operacionais
*   **0: helmet** (Capacete de segurança). Inclua todo o capacete. Se estiver na mão, *não* rotular (depende da regra de negócio, mas geralmente queremos "na cabeça").
*   **1: vest** (Colete reflexivo).
*   **2: person** (Pessoa). Inclua o corpo visível.

### Regras de Ouro da BBox
1.  **Tightness**: A caixa deve "tocar" as bordas do objeto. Não deixe muito ar, nem corte pedaços do objeto.
2.  **Oclusão**: Se o objeto está 50% tapado, rotule o que é visível (YOLO lida bem com isso). Se está 95% tapado e irreconhecível, ignore.
3.  **Consistência**: Se você rotula "cabeça sem capacete" como "person" em uma imagem, faça isso em todas. Não crie ambiguidades.

### Ferramentas Recomendadas
*   **LabelImg** (Local, clássico).
*   **CVAT** (Robustez profissional).
*   **Roboflow** (Web, fácil collab).



In [48]:
# Review Sampler Automático (Smart QA)
def review_sampler(dataset_path, split='train', top_k=6):
    img_dir = os.path.join(dataset_path, 'images', split)
    lbl_dir = os.path.join(dataset_path, 'labels', split)
    
    candidates = []
    
    for lbl_file in glob.glob(os.path.join(lbl_dir, "*.txt")):
        with open(lbl_file) as f:
            lines = f.readlines()
            
        n_objs = len(lines)
        has_micro = False
        has_edge = False
        
        for line in lines:
            _, x, y, w, h = map(float, line.split())
            if w*h < 0.005: has_micro = True 
            if x-w/2 < 0.01 or x+w/2 > 0.99: has_edge = True
            
        score = 0
        score += n_objs * 1      
        score += 50 if has_micro else 0  
        score += 10 if has_edge else 0   
        
        candidates.append((score, lbl_file))
        
    candidates.sort(key=lambda x: x[0], reverse=True)
    top_files = [os.path.splitext(os.path.basename(c[1]))[0] + ".jpg" for c in candidates[:top_k]]
    
    print(f"Revisando Top-{top_k} Casos Suspeitos (Crowded/Micro/Edge)...")
    
    plt.figure(figsize=(15, 8))
    for i, img_file in enumerate(top_files):
        img_path = os.path.join(img_dir, img_file)
        if not os.path.exists(img_path): continue
        
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        h_img, w_img, _ = img.shape
        lbl_path = os.path.join(lbl_dir, os.path.splitext(img_file)[0] + ".txt")
        
        with open(lbl_path) as f:
            for line in f:
                c, x, y, w, h = map(float, line.split())
                x1, y1 = int((x - w/2) * w_img), int((y - h/2) * h_img)
                x2, y2 = int((x + w/2) * w_img), int((y + h/2) * h_img)
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

        plt.subplot(2, 3, i+1)
        plt.imshow(img)
        # FIX: Escape newline for python string serialization
        plt.title(f"{img_file}\nScore: {candidates[i][0]}")
        plt.axis('off')
    plt.show()

# review_sampler(FINAL_DATASET_DIR)



## 8. Data Augmentations: Ensinando Robustez

O modelo não "entende" objetos; ele vê padrões de pixels. Se ele só viu capacetes amarelos à luz do dia, ele falhará em capacetes amarelos à noite.
**Augmentation** expande artificialmente o dataset variando as condições.

### Duas Estratégias

1.  **Online (On-the-fly via Ultralytics)**: O YOLO11 já faz muito augmentation pesado durante o treino (`mosaic`, `mixup`, `hsv`, `crop`, `flip`). Isso é configurável nos hiperparâmetros (arquivo `.yaml` ou args de treino).
    *   *Bom para*: Variações geométricas e de cor padrão.
2.  **Offline (Pré-processamento via Albumentations)**: Criar um dataset "v2" com efeitos específicos que o YOLO não faz nativamente ou controlar melhor a probabilidade.
    *   *Bom para*: Efeitos de clima (chuva/neblina), desfoque de movimento, etc.

Vamos focar em visualizar o **Pipeline Albumentations** para entender o que está acontecendo.



In [49]:
import albumentations as A

# Pipeline "Seguro" para Detecção
aug_pipeline = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.OneOf([
        A.MotionBlur(p=1),  
        A.GaussNoise(p=1),  
        A.Defocus(p=1),     
    ], p=0.3),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

print("Pipeline Albumentations criado.")



Pipeline Albumentations criado.


In [50]:
# Visualizando Augmentations
def visualize_augmentations(dataset_path, pipeline, samples=6):
    img_dir = os.path.join(dataset_path, 'images', 'train')
    lbl_dir = os.path.join(dataset_path, 'labels', 'train')
    
    file_list = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]
    if not file_list: return
    
    chosen = random.sample(file_list, min(samples, len(file_list)))
    
    plt.figure(figsize=(16, 4 * samples))
    
    for row, img_file in enumerate(chosen):
        img_path = os.path.join(img_dir, img_file)
        lbl_path = os.path.join(lbl_dir, os.path.splitext(img_file)[0] + ".txt")
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        
        bboxes = []
        labels = []
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r', encoding='utf-8') as f:
                for line in f:
                    c, x, y, w, h = map(float, line.split())
                    bboxes.append([x, y, w, h])
                    labels.append(int(c))
        
        plt.subplot(samples, 4, row*4 + 1)
        viz_img = image.copy()
        h_img, w_img, _ = viz_img.shape
        for bbox, cls in zip(bboxes, labels):
            x, y, w, h = bbox
            x1, y1 = int((x - w/2) * w_img), int((y - h/2) * h_img)
            x2, y2 = int((x + w/2) * w_img), int((y + h/2) * h_img)
            cv2.rectangle(viz_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        plt.imshow(viz_img)
        plt.title("Original")
        plt.axis('off')
        
        for col in range(3):
            try:
                augmented = pipeline(image=image, bboxes=bboxes, class_labels=labels)
                aug_img = augmented['image']
                aug_bboxes = augmented['bboxes']
                h_aug, w_aug, _ = aug_img.shape
                for bbox in aug_bboxes:
                    x, y, w, h = bbox
                    x1, y1 = int((x - w/2) * w_aug), int((y - h/2) * h_aug)
                    x2, y2 = int((x + w/2) * w_aug), int((y + h/2) * h_aug)
                    cv2.rectangle(aug_img, (x1, y1), (x2, y2), (255, 0, 255), 2)
                
                plt.subplot(samples, 4, row*4 + 2 + col)
                plt.imshow(aug_img)
                plt.title(f"Aug {col+1}")
                plt.axis('off')
            except Exception as e:
                plt.subplot(samples, 4, row*4 + 2 + col)
                # FIX: Escape newline for python string serialization
                plt.text(0.5, 0.5, f"Drop/Error\n{str(e)}", ha='center')
                plt.axis('off')

    plt.tight_layout()
    plt.show()

# visualize_augmentations(FINAL_DATASET_DIR, aug_pipeline)



### Riscos e Contra-indicações
Nem todo augmentation ajuda. Se você distorce demais, o modelo aprende lixo.

| Objetivo | Augmentation Sugerida | Risco / Cuidado |
| :--- | :--- | :--- |
| **Robustez a Luz** | Brightness, Contrast, Gamma | Se exagerar, apaga detalhes em áreas escuras. |
| **Robustez a Câmera** | Blur, Noise, Compression | Se o objeto for muito pequeno (ex: < 10px), o blur pode fazê-lo desaparecer. |
| **Robustez a Posição** | Shift, Crop, Scale | **Flip Vertical** quase nunca faz sentido para pessoas/capacetes (gravidade existe!). |
| **Generalização** | MixUp, Mosaic (Ultralytics) | Ótimo para treino, mas confuso visualmente para humanos. |



## 9. Treino Baseline (Transfer Learning)

Agora que conhecemos os dados, vamos rodar um modelo simples para ter uma **Linha de Base**.
Não tente tunar nada ainda. Use "defaults razoáveis".

### O que acontece no Fine-tuning?
Carregamos `yolov8n.pt` (treinado em COCO).
1.  **Backbone (Corpo)**: Detector de características universais (bordas, texturas). Mantemos (ou treinamos pouco).
2.  **Head (Cabeça)**: A última camada. Ela "sabia" detectar *Cachorro, Gato, Carro...*. Nós substituímos ela por uma nova que aprenderá *Capacete, Colete, Pessoa*.

### Monitorando Overfitting
*   **Good**: Loss de Treino cai, Loss de Validação cai.
*   **Overfit**: Loss de Treino cai MUITO, Loss de Validação começa a **SUBIR**. (O modelo decorou o treino).
*   **Underfit**: Loss de Treino não cai (ou cai muito devagar). (Modelo burro ou dados ruins).



In [51]:
from ultralytics import YOLO

# 1. Carregar Modelo Pré-treinado
# 'n' = Nano (mais rápido, menor acurácia). Bom para baseline.
model_baseline = YOLO('yolo11n.pt') 

print("Modelo carregado. Classes originais:", len(model_baseline.names))
# Nota: Ao iniciar o treino com um data.yaml diferente, o YOLO substitui automaticamente o Head.

RuntimeError: operator torchvision::nms does not exist

In [ ]:
# 2. Executar Treino Baseline
# Configuração Conservadora para RTX 4060 (8GB)

train_args = {
    'data': os.path.join(FINAL_DATASET_DIR, "data.yaml"),
    'epochs': 30,           # Baseline curto. Na prática use 50-100.
    'imgsz': 640,           # Tamanho padrão.
    'batch': 16,            # 16 costuma ser seguro para 8GB. Se der OOM, baixe para 8.
    'project': RUNS_DIR,    # Salvar em ./workspace/runs
    'name': 'baseline_v1',  # Nome da pasta do experimento
    'exist_ok': True,       # Sobrescrever se existir (cuidado!)
    'amp': True,            # Mixed Precision (rápido e menos memória)
    'cache': True,          # Cache RAM (se couber, acelera muito)
    'patience': 10,         # Early Stopping (para se estagnar por 10 epochs)
    'device': '0' if torch.cuda.is_available() else 'cpu',
    'verbose': True
}

print(f"Iniciando Treino Baseline em {train_args['device']}...")
results = model_baseline.train(**train_args)

print(f"Treino concluído. Resultados em: {results.save_dir}")



In [ ]:
# 3. Registrar Experimento
# É útil manter um log JSON próprio além do MLFlow/WandB para análises rápidas via Pandas.

import datetime

def log_experiment(name, results_obj, params):
    log_path = os.path.join(REPORTS_DIR, "experiments_log.json")
    
    # Extrair métricas chave
    metrics = {
        "map50": results_obj.box.map50,
        "map50-95": results_obj.box.map,
        "precision": results_obj.box.mp,
        "recall": results_obj.box.mr
    }
    
    entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "name": name,
        "metrics": metrics,
        "params": {k:str(v) for k,v in params.items()}, # Serialize
        "save_dir": str(results_obj.save_dir)
    }
    
    logs = []
    if os.path.exists(log_path):
        with open(log_path, 'r', encoding='utf-8') as f:
            try: logs = json.load(f)
            except: pass
            
    logs.append(entry)
    
    with open(log_path, 'w', encoding='utf-8') as f:
        json.dump(logs, f, indent=2)
        
    print(f"Experimento '{name}' registrado em {log_path}")
    return pd.DataFrame([entry])

# Logar o baseline
log_experiment("baseline_nano_30ep", results, train_args)



### Dicas Avançadas: Resume e Freeze

1.  **Resume**: Se a luz cair na epoch 29/30:
    ```python
    # model = YOLO("workspace/runs/baseline_v1/weights/last.pt")
    # model.train(resume=True)
    ```

2.  **Freeze**: Se você tiver pouquíssimos dados (< 100 img), pode ajudar congelar o backbone para não "quebrar" os pesos pré-treinados.
    ```python
    # model.train(data=..., freeze=10) # Congela as 10 primeiras camadas
    ```
    No YOLO11, `freeze` aceita um int (número de camadas) ou lista de índices.



## 10. Avaliação e Error Analysis

O `mAP` é apenas um número. Ele não te diz **onde** seu modelo está errando.
Vamos fazer o "Raio-X" do modelo baseline.

### Interpretando Métricas
*   **Precision (Precisão)**: De todas as detecções do modelo, quantas estão corretas? (Evita e-mails falsos para o chefe).
*   **Recall (Revocação)**: De todos os EPIs reais na imagem, quantos o modelo achou? (Evita acidentes não detectados).
*   **mAP50**: A "nota geral" da prova. Considera Precision e Recall combinados, aceitando caixas com >50% de sobreposição (IoU).

    > **Trade-off**: Geralmente, aumentar Precision diminui Recall e vice-versa. O mAP é o equilíbrio.


In [ ]:
# 1. Rodar Validação Oficial
# Isso gera as matrizes de confusão e curvas PR oficiais na pasta do experimento.
val_results = model_baseline.val(split='val')

print(f"Mean Average Precision (mAP50): {val_results.box.map50:.3f}")
print(f"Precision: {val_results.box.mp:.3f}")
print(f"Recall: {val_results.box.mr:.3f}")

In [ ]:
# 2. Plotar Curvas e Matrizes Geradas
from IPython.display import Image, display

# O YOLO salva plots automaticamente na pasta do 'run'. Vamos achá-la.
run_dir = str(val_results.save_dir) 

print(f"Exibindo gráficos salvos em: {run_dir}")

plots = [
    "confusion_matrix.png",
    "F1_curve.png",
    "PR_curve.png",
    "labels.jpg" # Mostra distribuição do GT
]

for p in plots:
    path = os.path.join(run_dir, p)
    if os.path.exists(path):
        display(Image(filename=path, width=600))
    else:
        print(f"Plot {p} não encontrado (pode exigir scikit-learn instalado).")



### Análise de Erros Visual (Dashboard)
Números agregados escondem a verdade. Vamos ver **lado a lado**:
*   Esquerda: **Ground Truth** (O que o humano marcou).
*   Direita: **Prediction** (O que o modelo viu).

Se o modelo vir algo na direita que não está na esquerda:
1.  O modelo está alucinando (Erro dele)? ou...
2.  O humano esqueceu de rotular (Erro nosso)? **(Muito comum!)**



In [ ]:
# Visualização Lado a Lado: GT vs Pred
def plot_errors(model, dataset_path, split='val', n=6, conf_threshold=0.25):
    img_dir = os.path.join(dataset_path, 'images', split)
    lbl_dir = os.path.join(dataset_path, 'labels', split)
    
    files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]
    if not files: return
    
    samples = random.sample(files, min(n, len(files)))
    
    plt.figure(figsize=(15, 5*len(samples)))
    
    for i, img_file in enumerate(samples):
        img_path = os.path.join(img_dir, img_file)
        lbl_path = os.path.join(lbl_dir, os.path.splitext(img_file)[0] + ".txt")
        
        # Leitura Imagem
        img_raw = cv2.imread(img_path)
        img_gt = img_raw.copy()
        img_pred = img_raw.copy()
        h, w, _ = img_raw.shape
        
        # --- ESQUERDA: GROUND TRUTH ---
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r', encoding='utf-8') as f:
                for line in f:
                    c, cx, cy, bw, bh = map(float, line.split())
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)
                    color = (0, 255, 0) # Verde = Verdade
                    cv2.rectangle(img_gt, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(img_gt, f"GT {int(c)}", (x1, y1-5), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # --- DIREITA: PREDICTIONS ---
        results = model.predict(img_path, conf=conf_threshold, verbose=False)
        for box in results[0].boxes:
            coords = box.xyxy[0].cpu().numpy() # x1, y1, x2, y2
            conf = float(box.conf)
            cls_id = int(box.cls)
            
            x1, y1, x2, y2 = map(int, coords)
            color = (0, 0, 255) # Vermelho = Prediction
            cv2.rectangle(img_pred, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img_pred, f"{model.names[cls_id]} {conf:.2f}", (x1, y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
        # Plot
        plt.subplot(len(samples), 2, i*2 + 1)
        plt.imshow(cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB))
        plt.title(f"Ground Truth: {img_file}")
        plt.axis('off')
        
        plt.subplot(len(samples), 2, i*2 + 2)
        plt.imshow(cv2.cvtColor(img_pred, cv2.COLOR_BGR2RGB))
        plt.title(f"Prediction (Conf > {conf_threshold})")
        plt.axis('off')
        
    plt.tight_layout()
    plt.show()

# plot_errors(model_baseline, FINAL_DATASET_DIR, split='val', n=6)



### Guia de Ação
*   **Se FN domina (Muitos objetos perdidos):**
    *   Falta de treino (mais epochs).
    *   Objetos muito pequenos (aumentar `imgsz` de 640 para 1280).
    *   Labels faltando no GT (o modelo aprendeu a ignorar).
*   **Se FP domina (Muitas alucinações):**
    *   Background ruidoso? (Adicionar imagens vazias `background` no treino ajuda).
    *   Aumentar threshold de confiança na inferência.



## 11. Bônus: Rastreamento com MLflow

O YOLOv8 integra-se nativamente com MLflow.
Não é necessário escrever callbacks complexos. Basta configurar o ambiente.

### Como usar o MLflow Dashboard
1.  Abra um terminal no VSCode (`Ctrl + '`).
2.  Ative seu ambiente (`myenv`).
3.  Rode: `mlflow ui --port 5000 --backend-store-uri ./workspace/mlruns`
4.  Acesse `http://localhost:5000` no navegador.



In [ ]:
import mlflow

# Configurar MLflow para salvar localmente na pasta do projeto
# Isso evita poluir seu diretório de usuário (e funciona no Colab/Kaggle igual).
mlflow_tracking_uri = os.path.join(BASE_DIR, "mlruns")
os.makedirs(mlflow_tracking_uri, exist_ok=True)

mlflow.set_tracking_uri(f"file:///{mlflow_tracking_uri}")
mlflow.set_experiment(PROJECT_NAME)

# O YOLOv8 detecta automaticamente o MLflow se ele estiver instalado e ativo.
# Mas podemos forçar o nome do run via variável de ambiente se quisermos.
os.environ["MLFLOW_TRACKING_URI"] = f"file:///{mlflow_tracking_uri}"
os.environ["MLFLOW_EXPERIMENT_NAME"] = PROJECT_NAME

print(f"MLflow configurado! Logs irão para: {mlflow_tracking_uri}")
print("Para ver o dashboard, rode no terminal:")
print(f"mlflow ui --port 5000 --backend-store-uri {mlflow_tracking_uri}")




# 8.5 — Migração para Dataset Real (PPE) + Validação

**OBJETIVO CRÍTICO**: Esta seção serve como um "Safety Check" antes de gastar horas treinando ou rodando HPO.
Até agora, é possível que tenhamos usado um **Dummy Dataset** (retângulos coloridos) apenas para testar o código.
Para ter resultados reais, precisamos garantir que o dataset `ppe_v1` contém fotos reais e labels corretos.

**O que vamos verificar:**
1.  **Configuração Atual**: Onde o `data.yaml` está apontando.
2.  **Volume de Dados**: Se tivermos < 50 imagens, algo está errado (dataset dummy ou incompleto).
3.  **Consistência Profunda**:
    *   Orphans (Imagem sem label ou Label sem imagem).
    *   Formato YOLO (5 colunas, classes válidas, coords normalizadas).
4.  **Prova Visual**: Vamos plotar imagens reais com suas bounding boxes para confirmação humana.


In [ ]:

import yaml

print("--- [CHECK 1] Current data.yaml Configuration ---")
yaml_path = os.path.join(FINAL_DATASET_DIR, "data.yaml")

if os.path.exists(yaml_path):
    with open(yaml_path, 'r', encoding='utf-8') as f:
        data_config = yaml.safe_load(f)
        
    print(f"Path: {data_config.get('path')}")
    print(f"Train: {data_config.get('train')}")
    print(f"Val: {data_config.get('val')}")
    print(f"Names: {data_config.get('names')}")
else:
    print(f"ERROR: {yaml_path} not found!")


In [ ]:

import glob

print("\n--- [CHECK 2] Dataset Volume (Real vs Dummy) ---")

train_img_dir = os.path.join(FINAL_DATASET_DIR, "images", "train")
val_img_dir = os.path.join(FINAL_DATASET_DIR, "images", "val")

n_train = len(glob.glob(os.path.join(train_img_dir, "*.jpg"))) + len(glob.glob(os.path.join(train_img_dir, "*.png")))
n_val = len(glob.glob(os.path.join(val_img_dir, "*.jpg"))) + len(glob.glob(os.path.join(val_img_dir, "*.png")))

print(f"Training Images: {n_train}")
print(f"Validation Images: {n_val}")

if n_train < 100 or n_val < 20:
    print("\n" + "="*60)
    print("WARNING: DATASET MULTI-SMALL DETECTED!")
    print("Isso se parece com um Dummy Dataset ou um teste muito pequeno.")
    print("Se você espera resultados reais, por favor extraia o dataset completo.")
    print("="*60)
else:
    print("Volume parece razoável para um dataset real (ou small subset).")


In [ ]:

print("\n--- [CHECK 3] Consistency & Sanitization ---")

def sanitize_split(split_name):
    img_path = os.path.join(FINAL_DATASET_DIR, "images", split_name)
    lbl_path = os.path.join(FINAL_DATASET_DIR, "labels", split_name)
    quarantine_path = os.path.join(FINAL_DATASET_DIR, "_quarantine", split_name)
    
    os.makedirs(quarantine_path, exist_ok=True)
    
    if not os.path.exists(img_path) or not os.path.exists(lbl_path):
        print(f"[{split_name}] Directory not found. Skipping.")
        return

    # Get basenames
    imgs = {os.path.splitext(f)[0] for f in os.listdir(img_path) if f.endswith(('.jpg', '.png'))}
    lbls = {os.path.splitext(f)[0] for f in os.listdir(lbl_path) if f.endswith('.txt')}
    
    # 1. Orphans
    img_orphans = imgs - lbls
    lbl_orphans = lbls - imgs
    
    if img_orphans:
        print(f"[{split_name}] Found {len(img_orphans)} images without labels. Moving to quarantine...")
        for name in img_orphans:
            # find extension
            full_name = [f for f in os.listdir(img_path) if f.startswith(name)][0]
            shutil.move(os.path.join(img_path, full_name), os.path.join(quarantine_path, full_name))
            
    if lbl_orphans:
        print(f"[{split_name}] Found {len(lbl_orphans)} labels without images. Moving to quarantine...")
        for name in lbl_orphans:
            shutil.move(os.path.join(lbl_path, name + ".txt"), os.path.join(quarantine_path, name + ".txt"))

    # 2. Format Val
    valid_count, bad_count = 0, 0
    
    # Refresh lists after orphan removal
    lbls = [f for f in os.listdir(lbl_path) if f.endswith('.txt')]
    
    for lbl_file in lbls:
        is_bad = False
        fpath = os.path.join(lbl_path, lbl_file)
        with open(fpath, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    is_bad = True; break
                
                try:
                    cls = int(parts[0])
                    coords = [float(x) for x in parts[1:]]
                    if not (0 <= cls <= 2): # Hardcoded helper for PPE (3 classes)
                        is_bad = True; break
                    if any(c < 0 or c > 1 for c in coords):
                        is_bad = True; break
                except ValueError:
                    is_bad = True; break
        
        if is_bad:
            print(f"Corrupt label found: {lbl_file}. Moving to quarantine.")
            shutil.move(fpath, os.path.join(quarantine_path, lbl_file))
            # Also move image
            img_cand = [f for f in os.listdir(img_path) if f.startswith(os.path.splitext(lbl_file)[0])]
            if img_cand:
                shutil.move(os.path.join(img_path, img_cand[0]), os.path.join(quarantine_path, img_cand[0]))
            bad_count += 1
        else:
            valid_count += 1
            
    print(f"[{split_name}] Valid Pairs: {valid_count} | Moved to Quarantine: {len(img_orphans) + len(lbl_orphans) + bad_count}")

sanitize_split('train')
sanitize_split('val')


In [ ]:

print("\n--- [CHECK 4] Visual Sanity Check (Is this Real Data?) ---")

def plot_random_grid(split='train', num=9):
    img_dir = os.path.join(FINAL_DATASET_DIR, 'images', split)
    lbl_dir = os.path.join(FINAL_DATASET_DIR, 'labels', split)
    
    if not os.path.exists(img_dir):
        print(f"Directory {img_dir} does not exist.")
        return

    all_imgs = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
    if not all_imgs:
        print(f"No images in {split} to plot.")
        return
        
    selected = random.sample(all_imgs, min(num, len(all_imgs)))
    
    plt.figure(figsize=(15, 10))
    for i, img_file in enumerate(selected):
        img_path = os.path.join(img_dir, img_file)
        lbl_path = os.path.join(lbl_dir, os.path.splitext(img_file)[0] + ".txt")
        
        img = cv2.imread(img_path)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape
        
        if os.path.exists(lbl_path):
            with open(lbl_path, 'r', encoding='utf-8') as f:
                for line in f:
                    parts = list(map(float, line.split()))
                    if len(parts) == 5:
                        c, cx, cy, bw, bh = parts
                        x1 = int((cx - bw/2) * w)
                        y1 = int((cy - bh/2) * h)
                        x2 = int((cx + bw/2) * w)
                        y2 = int((cy + bh/2) * h)
                        
                        color = (0, 255, 0) # Green for annotations
                        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(img, str(int(c)), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{split}/{img_file}")
        
    plt.tight_layout()
    plt.show()

print("Plotting Training Samples...")
plot_random_grid('train')

print("Plotting Validation Samples...")
plot_random_grid('val')


In [ ]:

print("\n--- [CHECK 5] Class Statistics ---")
import pandas as pd
import seaborn as sns

def get_class_stats(split):
    lbl_dir = os.path.join(FINAL_DATASET_DIR, 'labels', split)
    if not os.path.exists(lbl_dir): return {}
    
    counts = {0:0, 1:0, 2:0}
    
    for lfile in os.listdir(lbl_dir):
        if not lfile.endswith('.txt'): continue
        with open(os.path.join(lbl_dir, lfile)) as f:
            for line in f:
                parts = line.split()
                if not parts: continue
                c = int(parts[0])
                if c in counts: counts[c] += 1
    return counts

stats_train = get_class_stats('train')
stats_val = get_class_stats('val')

if stats_train and stats_val:
    df_stats = pd.DataFrame([stats_train, stats_val], index=['Train', 'Val']).T
    print(df_stats)

    # Warning for zero classes
    if (df_stats == 0).any().any():
        print("\nCRITICAL WARNING: Uma ou mais classes têm ZERO exemplos. Isso quebrará o treino.")
else:
    print("Could not generate statistics (missing directories).")


In [ ]:

print("\n--- [CHECK 6] Finalizing configuration ---")

# Re-write data.yaml to be absolutely sure
yaml_content = f'''
path: {FINAL_DATASET_DIR}
train: images/train
val: images/val
test: 

names:
  0: helmet
  1: vest
  2: person
'''
with open(os.path.join(FINAL_DATASET_DIR, "data.yaml"), 'w') as f:
    f.write(yaml_content)

print(f"data.yaml atualizado e verificado em: {FINAL_DATASET_DIR}/data.yaml")
print("Checklist:")
print("[ ] Estrutura OK")
print(f"[ ] Val Count: {n_val}")
print("[ ] Visual Check (acima)")
